In [1]:
%load_ext autoreload
%autoreload 2

# COMPUTE AND SAVE MULTI VIEW GRAD CAM

In [3]:
from train_multiview_network import TrainMultiInputModel
from multi_view_classification_network import MultiInputModel
from data_loader import CustomDataSet, MultiInputLoader
from testMV_GC import GradCam
from visualisation_utils import image_net_postprocessing, NormalizeInverse
from config import get_config, get_aty_data_config, get_ext_data_config
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torchvision import datasets
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, accuracy_score
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image
from pytorch_grad_cam_adapt.activations_and_gradients import ActivationsAndGradients
from pytorch_grad_cam_adapt.image import scale_cam_image, tensor2cam, subplot, vis_outs2images_classes,image2cam,tensor2img, show_cam_on_image
from pytorch_grad_cam_adapt.model_targets import ClassifierOutputTarget
from pytorch_grad_cam_adapt.grad_cam import GradCAM
from pytorch_grad_cam_adapt.guided_backprop import GuidedBackpropReLUModel

# Save Grad CAM 

In [4]:
torch.cuda.empty_cache()
model_config, data_config = get_config()
trainer = TrainMultiInputModel(model_config,data_config)
model_recto = model_config['model_recto']
model_verso =   model_config['model_verso']            
model = MultiInputModel(model_recto,model_verso,pretrained=True, fine_tune=model_config['fine_tune'], num_classes=7)
model_name = 'model_recto_ResNet50_model_verso_ResNet50_epochs_30_lr_0.001_batch_size_32_pretrained_True_fine_tune_False'
checkpoint = torch.load(f'{trainer.main_path}/outputs/'+model_name+'.pth', map_location="cuda")
print('Loading trained model weights...')
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(trainer.device)

lb=["Alt","Big","Mac","Mil","Myc","Pse","Syl"]
le = preprocessing.LabelEncoder()
le.fit(lb) 

[INFO]: Loading pre-trained weights
[INFO]: Freezing hidden layers...
[INFO]: Loading pre-trained weights
[INFO]: Freezing hidden layers...
Loading trained model weights...


LabelEncoder()

In [9]:
target_layers = [[model.modelRecto_features.layer4],[model.modelVerso_features.layer4]]
gb_model = GuidedBackpropReLUModel(model=model, use_cuda=True)
gc_model = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

### Retrieve path

#### Typique

In [10]:
datasetRecto_test_datafolder = datasets.ImageFolder(
           trainer.loader.root_dir +'RECTO', 
            #transform=(self.get_test_transform())
        )
datasetVerso_test_datafolder = datasets.ImageFolder(
           trainer.loader.root_dir +'VERSO', 
            #transform=(self.get_test_transform())
        )

In [11]:
PATH_TESTS_DATASET_RECTO = []
PATH_TESTS_DATASET_VERSO = []
indices = trainer.datasetRecto_test.indices
for ind in indices:
    PATH_TESTS_DATASET_RECTO.append(os.path.basename(datasetRecto_test_datafolder.samples[ind][0]))
    PATH_TESTS_DATASET_VERSO.append(os.path.basename(datasetVerso_test_datafolder.samples[ind][0]))

#### Aty & Ext

In [12]:
data_aty_config = get_aty_data_config()
data_ext_config = get_ext_data_config()
aty_loader = MultiInputLoader(data_aty_config)
atydatasetRecto, atydatasetVerso = aty_loader.get_datasets()

ext_loader = MultiInputLoader(data_ext_config)
extdatasetRecto, extdatasetVerso = ext_loader.get_datasets()

In [13]:
PATH_ATY_DATASET_RECTO = [os.path.basename(path) for path in atydatasetRecto.all_imgs]
PATH_ATY_DATASET_VERSO = [os.path.basename(path) for path in atydatasetVerso.all_imgs]
PATH_EXT_DATASET_RECTO = [os.path.basename(path) for path in extdatasetRecto.all_imgs]
PATH_EXT_DATASET_VERSO = [os.path.basename(path) for path in extdatasetVerso.all_imgs]

### Save Image function

In [18]:
print(model_name)

model_recto_ResNet50_model_verso_ResNet50_epochs_30_lr_0.001_batch_size_32_pretrained_True_fine_tune_False


In [19]:
if not os.path.exists(os.path.join("/home","onyxia","work","gradcam_images",model_name)):
    os.mkdir(os.path.join("/","home","onyxia","work","gradcam_images","model_recto_ResNet50_model_verso_ResNet50_epochs_30_lr_0.001_batch_size_32_pretrained_True_fine_tune_False"))

In [8]:
if not os.path.exists(os.path.join("/home","onyxia","work","gradcam_images","ext_for_report",model_name)):
    os.mkdir(os.path.join("/","home","onyxia","work","gradcam_images","ext_for_report","model_recto_ResNet50_model_verso_ResNet50_epochs_30_lr_0.001_batch_size_32_pretrained_True_fine_tune_False"))

In [14]:
def compute_and_save_gradcam(input_recto,input_verso,filenames):
    gc_filenames = [os.path.join("/home","onyxia","work","gradcam_images",model_name,os.path.splitext(filenames[0])[0]+"_GRAD_CAM.jpeg"),
                    os.path.join("/home","onyxia","work","gradcam_images",model_name,os.path.splitext(filenames[1])[0]+"_GRAD_CAM.jpeg")]
    cam_gb_filenames = [os.path.join("/home","onyxia","work","gradcam_images",model_name,os.path.splitext(filenames[0])[0]+"_GUIDED_GRAD_CAM.jpeg"),
                        os.path.join("/home","onyxia","work","gradcam_images",model_name,os.path.splitext(filenames[1])[0]+"_GUIDED_GRAD_CAM.jpeg")]
    gb = gb_model([input_recto,input_verso], target_category=None)
    gc = gc_model(input_tensors = [input_recto, input_verso],targets = None)
    recto_cam = gc[0][0][0]
    recto_gb = gb[0]

    verso_cam = gc[0][1][0]
    verso_gb = gb[1]

    recto_cam_gb = np.zeros(shape=(224,224,3))
    verso_cam_gb = np.zeros(shape=(224,224,3))
    for i in range(3):                  
        recto_cam_gb[:,:,i] = np.multiply(recto_gb[:,:,i],recto_cam)
        recto_cam_gb[:,:,i] = recto_cam_gb[:,:,i] - np.min(recto_cam_gb[:,:,i])
        recto_cam_gb[:,:,i] = recto_cam_gb[:,:,i] / (1e-7 + np.max(recto_cam_gb[:,:,i]))
        recto_cam_gb[:,:,i] = np.float32(recto_cam_gb[:,:,i])
        verso_cam_gb[:,:,i] = np.multiply(verso_gb[:,:,i],verso_cam)
        verso_cam_gb[:,:,i] = verso_cam_gb[:,:,i] - np.min(verso_cam_gb[:,:,i])
        verso_cam_gb[:,:,i] = verso_cam_gb[:,:,i] / (1e-7 + np.max(verso_cam_gb[:,:,i]))
        verso_cam_gb[:,:,i] = np.float32(verso_cam_gb[:,:,i])
        
    recto_cam_gb = cv2.cvtColor(recto_cam_gb.astype('float32'),cv2.COLOR_BGR2RGB)
    verso_cam_gb = cv2.cvtColor(verso_cam_gb.astype('float32'),cv2.COLOR_BGR2RGB)
    recto_gc = image2cam(image_net_postprocessing(input_recto),recto_cam)
    verso_gc = image2cam(image_net_postprocessing(input_verso),verso_cam)
    recto_gc = cv2.cvtColor(recto_gc.astype('float32'),cv2.COLOR_BGR2RGB)
    verso_gc = cv2.cvtColor(verso_gc.astype('float32'),cv2.COLOR_BGR2RGB)
    #Save Image
    cv2.imwrite(gc_filenames[0],np.uint8(recto_gc*255))
    cv2.imwrite(gc_filenames[1],np.uint8(verso_gc*255))
    cv2.imwrite(cam_gb_filenames[0],np.uint8(recto_cam_gb*255))
    cv2.imwrite(cam_gb_filenames[1],np.uint8(verso_cam_gb*255))
    return gc_filenames, cam_gb_filenames

### Typical test images 

In [16]:
from torch.nn import functional as F
model.eval()
counter=0
outputs_test_preds= []
outputs_test_probs = []
real_labels = []
filename_recto_gc = []
filename_verso_gc = []
filename_recto_cam_gb = []
filename_verso_cam_gb = []
model.eval()
for i, (recto,verso) in tqdm(enumerate(zip(trainer.datasetRecto_test,trainer.datasetVerso_test)), total=len(trainer.datasetRecto_test)):
    with torch.no_grad():
        counter += 1
        image_recto, label = recto
        image_recto = image_recto.unsqueeze(0).to(trainer.device)
        image_verso, _ = verso
        image_verso = image_verso.unsqueeze(0).to(trainer.device)
        # Forward pass.
        outputs_test = model([image_recto,image_verso])
        outputs_test_prob = F.softmax(outputs_test,dim=1)
        prob , pred = torch.max(outputs_test_prob.data, 1)
        real_labels.append(label)
        outputs_test_preds.append(pred)
        outputs_test_probs.append(prob)
    # GRAD CAM STUFF
    gc_filenames, cam_gb_filenames = compute_and_save_gradcam(image_recto,image_verso,[PATH_TESTS_DATASET_RECTO[i],PATH_TESTS_DATASET_VERSO[i]])
    filename_recto_gc.append(os.path.basename(gc_filenames[0]))
    filename_verso_gc.append(os.path.basename(gc_filenames [1]))
    filename_recto_cam_gb.append(os.path.basename(cam_gb_filenames[0]))
    filename_verso_cam_gb.append(os.path.basename(cam_gb_filenames[1]))

100%|██████████| 2885/2885 [20:09<00:00,  2.38it/s]


In [68]:
trainer.model = model

In [17]:
df_results_test = pd.DataFrame({"path_test_data_recto": PATH_TESTS_DATASET_RECTO,
                                "path_test_data_verso" : PATH_TESTS_DATASET_VERSO,
                                "outputs_test_preds" : [out.cpu().numpy()[0] for out in outputs_test_preds],
                                "outputs_test_probs" : [out.cpu().numpy()[0] for out in outputs_test_probs],
                                "real_labels" : real_labels,
                                "filename_recto_gc" : filename_recto_gc,
                                "filename_verso_gc" : filename_verso_gc,
                                "filename_recto_cam_gb" : filename_recto_cam_gb,
                                "filename_verso_cam_gb" : filename_verso_cam_gb})
                                

In [19]:
df_results_test.head(10)

,path_test_data_recto,path_test_data_verso,outputs_test_preds,outputs_test_probs,real_labels,filename_recto_gc,filename_verso_gc,filename_recto_cam_gb,filename_verso_cam_gb
0,Typ_Mac_Rha_1015_Recto.jpeg,Typ_Mac_Rha_1015_Verso.jpeg,5,0.732424,2,Typ_Mac_Rha_1015_Recto_GRAD_CAM.jpeg,Typ_Mac_Rha_1015_Verso_GRAD_CAM.jpeg,Typ_Mac_Rha_1015_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mac_Rha_1015_Verso_GUIDED_GRAD_CAM.jpeg
1,Typ_Mil_Rha_4564_Recto.jpeg,Typ_Mil_Rha_4564_Verso.jpeg,3,0.999790,3,Typ_Mil_Rha_4564_Recto_GRAD_CAM.jpeg,Typ_Mil_Rha_4564_Verso_GRAD_CAM.jpeg,Typ_Mil_Rha_4564_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mil_Rha_4564_Verso_GUIDED_GRAD_CAM.jpeg
2,Typ_Big_Rhe_1406_Recto.jpeg,Typ_Big_Rhe_1406_Verso.jpeg,5,0.483821,1,Typ_Big_Rhe_1406_Recto_GRAD_CAM.jpeg,Typ_Big_Rhe_1406_Verso_GRAD_CAM.jpeg,Typ_Big_Rhe_1406_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Big_Rhe_1406_Verso_GUIDED_GRAD_CAM.jpeg
3,Typ_Syl_Rha_0209_Recto.jpeg,Typ_Syl_Rha_0209_Verso.jpeg,6,0.999817,6,Typ_Syl_Rha_0209_Recto_GRAD_CAM.jpeg,Typ_Syl_Rha_0209_Verso_GRAD_CAM.jpeg,Typ_Syl_Rha_0209_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Syl_Rha_0209_Verso_GUIDED_GRAD_CAM.jpeg
4,Typ_Mil_Rha_5016_Recto.jpeg,Typ_Mil_Rha_5016_Verso.jpeg,3,0.999995,3,Typ_Mil_Rha_5016_Recto_GRAD_CAM.jpeg,Typ_Mil_Rha_5016_Verso_GRAD_CAM.jpeg,Typ_Mil_Rha_5016_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mil_Rha_5016_Verso_GUIDED_GRAD_CAM.jpeg
5,Typ_Syl_Rha_0500_Recto.jpeg,Typ_Syl_Rha_0500_Verso.jpeg,6,0.999999,6,Typ_Syl_Rha_0500_Recto_GRAD_CAM.jpeg,Typ_Syl_Rha_0500_Verso_GRAD_CAM.jpeg,Typ_Syl_Rha_0500_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Syl_Rha_0500_Verso_GUIDED_GRAD_CAM.jpeg
6,Typ_Pse_Rha_1773_Recto.jpeg,Typ_Pse_Rha_1773_Verso.jpeg,5,0.961986,5,Typ_Pse_Rha_1773_Recto_GRAD_CAM.jpeg,Typ_Pse_Rha_1773_Verso_GRAD_CAM.jpeg,Typ_Pse_Rha_1773_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Pse_Rha_1773_Verso_GUIDED_GRAD_CAM.jpeg
7,Typ_Mac_Rhe_0148_Recto.jpeg,Typ_Mac_Rhe_0148_Verso.jpeg,2,0.998216,2,Typ_Mac_Rhe_0148_Recto_GRAD_CAM.jpeg,Typ_Mac_Rhe_0148_Verso_GRAD_CAM.jpeg,Typ_Mac_Rhe_0148_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mac_Rhe_0148_Verso_GUIDED_GRAD_CAM.jpeg
8,Typ_Mil_Rha_4616_Recto.jpeg,Typ_Mil_Rha_4616_Verso.jpeg,3,0.999988,3,Typ_Mil_Rha_4616_Recto_GRAD_CAM.jpeg,Typ_Mil_Rha_4616_Verso_GRAD_CAM.jpeg,Typ_Mil_Rha_4616_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mil_Rha_4616_Verso_GUIDED_GRAD_CAM.jpeg
9,Typ_Mil_Rha_4926_Recto.jpeg,Typ_Mil_Rha_4926_Verso.jpeg,3,0.999725,3,Typ_Mil_Rha_4926_Recto_GRAD_CAM.jpeg,Typ_Mil_Rha_4926_Verso_GRAD_CAM.jpeg,Typ_Mil_Rha_4926_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mil_Rha_4926_Verso_GUIDED_GRAD_CAM.jpeg


In [20]:
df_results_test["outputs_test_preds"] = df_results_test["outputs_test_preds"].apply(lambda x : le.inverse_transform([x])[0])
df_results_test["real_labels"] = df_results_test["real_labels"].apply(lambda x : le.inverse_transform([x])[0])

In [21]:
df_results_test.head(10)

,path_test_data_recto,path_test_data_verso,outputs_test_preds,outputs_test_probs,real_labels,filename_recto_gc,filename_verso_gc,filename_recto_cam_gb,filename_verso_cam_gb
0,Typ_Mac_Rha_1015_Recto.jpeg,Typ_Mac_Rha_1015_Verso.jpeg,Pse,0.732424,Mac,Typ_Mac_Rha_1015_Recto_GRAD_CAM.jpeg,Typ_Mac_Rha_1015_Verso_GRAD_CAM.jpeg,Typ_Mac_Rha_1015_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mac_Rha_1015_Verso_GUIDED_GRAD_CAM.jpeg
1,Typ_Mil_Rha_4564_Recto.jpeg,Typ_Mil_Rha_4564_Verso.jpeg,Mil,0.999790,Mil,Typ_Mil_Rha_4564_Recto_GRAD_CAM.jpeg,Typ_Mil_Rha_4564_Verso_GRAD_CAM.jpeg,Typ_Mil_Rha_4564_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mil_Rha_4564_Verso_GUIDED_GRAD_CAM.jpeg
2,Typ_Big_Rhe_1406_Recto.jpeg,Typ_Big_Rhe_1406_Verso.jpeg,Pse,0.483821,Big,Typ_Big_Rhe_1406_Recto_GRAD_CAM.jpeg,Typ_Big_Rhe_1406_Verso_GRAD_CAM.jpeg,Typ_Big_Rhe_1406_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Big_Rhe_1406_Verso_GUIDED_GRAD_CAM.jpeg
3,Typ_Syl_Rha_0209_Recto.jpeg,Typ_Syl_Rha_0209_Verso.jpeg,Syl,0.999817,Syl,Typ_Syl_Rha_0209_Recto_GRAD_CAM.jpeg,Typ_Syl_Rha_0209_Verso_GRAD_CAM.jpeg,Typ_Syl_Rha_0209_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Syl_Rha_0209_Verso_GUIDED_GRAD_CAM.jpeg
4,Typ_Mil_Rha_5016_Recto.jpeg,Typ_Mil_Rha_5016_Verso.jpeg,Mil,0.999995,Mil,Typ_Mil_Rha_5016_Recto_GRAD_CAM.jpeg,Typ_Mil_Rha_5016_Verso_GRAD_CAM.jpeg,Typ_Mil_Rha_5016_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mil_Rha_5016_Verso_GUIDED_GRAD_CAM.jpeg
5,Typ_Syl_Rha_0500_Recto.jpeg,Typ_Syl_Rha_0500_Verso.jpeg,Syl,0.999999,Syl,Typ_Syl_Rha_0500_Recto_GRAD_CAM.jpeg,Typ_Syl_Rha_0500_Verso_GRAD_CAM.jpeg,Typ_Syl_Rha_0500_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Syl_Rha_0500_Verso_GUIDED_GRAD_CAM.jpeg
6,Typ_Pse_Rha_1773_Recto.jpeg,Typ_Pse_Rha_1773_Verso.jpeg,Pse,0.961986,Pse,Typ_Pse_Rha_1773_Recto_GRAD_CAM.jpeg,Typ_Pse_Rha_1773_Verso_GRAD_CAM.jpeg,Typ_Pse_Rha_1773_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Pse_Rha_1773_Verso_GUIDED_GRAD_CAM.jpeg
7,Typ_Mac_Rhe_0148_Recto.jpeg,Typ_Mac_Rhe_0148_Verso.jpeg,Mac,0.998216,Mac,Typ_Mac_Rhe_0148_Recto_GRAD_CAM.jpeg,Typ_Mac_Rhe_0148_Verso_GRAD_CAM.jpeg,Typ_Mac_Rhe_0148_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mac_Rhe_0148_Verso_GUIDED_GRAD_CAM.jpeg
8,Typ_Mil_Rha_4616_Recto.jpeg,Typ_Mil_Rha_4616_Verso.jpeg,Mil,0.999988,Mil,Typ_Mil_Rha_4616_Recto_GRAD_CAM.jpeg,Typ_Mil_Rha_4616_Verso_GRAD_CAM.jpeg,Typ_Mil_Rha_4616_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mil_Rha_4616_Verso_GUIDED_GRAD_CAM.jpeg
9,Typ_Mil_Rha_4926_Recto.jpeg,Typ_Mil_Rha_4926_Verso.jpeg,Mil,0.999725,Mil,Typ_Mil_Rha_4926_Recto_GRAD_CAM.jpeg,Typ_Mil_Rha_4926_Verso_GRAD_CAM.jpeg,Typ_Mil_Rha_4926_Recto_GUIDED_GRAD_CAM.jpeg,Typ_Mil_Rha_4926_Verso_GUIDED_GRAD_CAM.jpeg


In [26]:
df_results_test.to_csv("/home/onyxia/work/pfe-deep-learning-maladies-plantes/multi_vue_pytorch/outputs/"+model_name+"_df_results_test.csv")

In [74]:
preds, probs, _ = trainer.predict(trainer.testRectoloader,trainer.testVersoloader)

  0%|          | 0/91 [00:00<?, ?it/s]

In [75]:
print(preds[:10],probs[:10])

[2, 3, 1, 6, 3, 6, 5, 2, 3, 3] [0.5339010953903198, 0.895734965801239, 0.5230067372322083, 0.9999940395355225, 0.9792180061340332, 1.0, 0.9384658932685852, 0.9912988543510437, 0.9873083233833313, 0.9702541828155518]


In [44]:
df_results_test['outputs_test_preds'].value_counts().sum()

2885

In [43]:
accuracy_score(df_results_test['real_labels'],df_results_test['outputs_test_preds'])

0.9518197573656846

### Aty Images

In [27]:
from torch.nn import functional as F
model.eval()
counter=0
outputs_aty_preds= []
outputs_aty_probs = []
filename_aty_recto_gc = []
filename_aty_verso_gc = []
filename_aty_recto_cam_gb = []
filename_aty_verso_cam_gb = []
model.eval()
for i, (recto,verso) in tqdm(enumerate(zip(atydatasetRecto,atydatasetVerso)), total=len(atydatasetRecto)):
    with torch.no_grad():
        counter += 1
        image_recto = recto
        image_recto = image_recto.unsqueeze(0).to(trainer.device)
        image_verso = verso
        image_verso = image_verso.unsqueeze(0).to(trainer.device)
        # Forward pass.
        outputs_aty = model([image_recto,image_verso])
        outputs_aty_prob = F.softmax(outputs_aty,dim=1)
        prob , pred = torch.max(outputs_aty_prob.data, 1)
        real_labels.append(label)
        outputs_aty_preds.append(pred)
        outputs_aty_probs.append(prob)
    # GRAD CAM STUFF
    gc_filenames, cam_gb_filenames = compute_and_save_gradcam(image_recto,image_verso,[PATH_ATY_DATASET_RECTO[i],PATH_ATY_DATASET_VERSO[i]])
    filename_aty_recto_gc.append(os.path.basename(gc_filenames[0]))
    filename_aty_verso_gc.append(os.path.basename(gc_filenames [1]))
    filename_aty_recto_cam_gb.append(os.path.basename(cam_gb_filenames[0]))
    filename_aty_verso_cam_gb.append(os.path.basename(cam_gb_filenames[1]))

100%|██████████| 649/649 [03:26<00:00,  3.14it/s]


In [28]:
df_results_aty = pd.DataFrame({"path_aty_data_recto": PATH_ATY_DATASET_RECTO,
                                "path_aty_data_verso" : PATH_ATY_DATASET_VERSO,
                                "outputs_aty_preds" : [out.cpu().numpy()[0] for out in outputs_aty_preds],
                                "outputs_aty_probs" : [out.cpu().numpy()[0] for out in outputs_aty_probs],
                                "filename_aty_recto_gc" : filename_aty_recto_gc,
                                "filename_aty_verso_gc" : filename_aty_verso_gc,
                                "filename_aty_recto_cam_gb" : filename_aty_recto_cam_gb,
                                "filename_verso_cam_gb" : filename_aty_verso_cam_gb})
                                

In [29]:
df_results_aty["outputs_aty_preds"] = df_results_aty["outputs_aty_preds"].apply(lambda x : le.inverse_transform([x])[0])

In [30]:
df_results_aty.to_csv("/home/onyxia/work/pfe-deep-learning-maladies-plantes/multi_vue_pytorch/outputs/"+model_name+"_df_results_aty.csv")

### Ext Images

In [27]:
from torch.nn import functional as F
model.eval()
counter=0
outputs_ext_preds= []
outputs_ext_probs = []
filename_ext_recto_gc = []
filename_ext_verso_gc = []
filename_ext_recto_cam_gb = []
filename_ext_verso_cam_gb = []
model.eval()
for i, (recto,verso) in tqdm(enumerate(zip(extdatasetRecto,extdatasetVerso)), total=len(extdatasetRecto)):
    with torch.no_grad():
        counter += 1
        image_recto = recto
        image_recto = image_recto.unsqueeze(0).to(trainer.device)
        image_verso = verso
        image_verso = image_verso.unsqueeze(0).to(trainer.device)
        # Forward pass.
        outputs_ext = model([image_recto,image_verso])
        outputs_ext_prob = F.softmax(outputs_ext,dim=1)
        prob , pred = torch.max(outputs_ext_prob.data, 1)
        real_labels.append(label)
        outputs_ext_preds.append(pred)
        outputs_ext_probs.append(prob)
    # GRAD CAM STUFF
    gc_filenames, cam_gb_filenames = compute_and_save_gradcam(image_recto,image_verso,[PATH_EXT_DATASET_RECTO[i],PATH_EXT_DATASET_VERSO[i]])
    filename_ext_recto_gc.append(os.path.basename(gc_filenames[0]))
    filename_ext_verso_gc.append(os.path.basename(gc_filenames [1]))
    filename_ext_recto_cam_gb.append(os.path.basename(cam_gb_filenames[0]))
    filename_ext_verso_cam_gb.append(os.path.basename(cam_gb_filenames[1]))

100%|██████████| 394/394 [02:57<00:00,  2.23it/s]


In [28]:
df_results_ext = pd.DataFrame({"path_ext_data_recto": PATH_EXT_DATASET_RECTO,
                                "path_ext_data_verso" : PATH_EXT_DATASET_VERSO,
                                "outputs_ext_preds" : [out.cpu().numpy()[0] for out in outputs_ext_preds],
                                "outputs_ext_probs" : [out.cpu().numpy()[0] for out in outputs_ext_probs],
                                "filename_ext_recto_gc" : filename_ext_recto_gc,
                                "filename_ext_verso_gc" : filename_ext_verso_gc,
                                "filename_ext_recto_cam_gb" : filename_ext_recto_cam_gb,
                                "filename_verso_cam_gb" : filename_ext_verso_cam_gb})

In [29]:
df_results_ext["outputs_ext_preds"] = df_results_ext["outputs_ext_preds"].apply(lambda x : le.inverse_transform([x])[0])
df_results_ext.to_csv("/home/onyxia/work/pfe-deep-learning-maladies-plantes/multi_vue_pytorch/outputs/"+model_name+"_df_results_ext.csv")

In [32]:
df_results_ext.head(20)

,path_ext_data_recto,path_ext_data_verso,outputs_ext_preds,outputs_ext_probs,filename_ext_recto_gc,filename_ext_verso_gc,filename_ext_recto_cam_gb,filename_verso_cam_gb
0,Ext_Bio_0020_Recto.jpeg,Ext_Bio_0020_Verso.jpeg,Alt,0.870731,Ext_Bio_0020_Recto_GRAD_CAM.jpeg,Ext_Bio_0020_Verso_GRAD_CAM.jpeg,Ext_Bio_0020_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Bio_0020_Verso_GUIDED_GRAD_CAM.jpeg
1,Ext_Bio_0035_Recto.jpeg,Ext_Bio_0035_Verso.jpeg,Syl,0.994071,Ext_Bio_0035_Recto_GRAD_CAM.jpeg,Ext_Bio_0035_Verso_GRAD_CAM.jpeg,Ext_Bio_0035_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Bio_0035_Verso_GUIDED_GRAD_CAM.jpeg
2,Ext_Bio_0036_Recto.jpeg,Ext_Bio_0036_Verso.jpeg,Mac,0.437899,Ext_Bio_0036_Recto_GRAD_CAM.jpeg,Ext_Bio_0036_Verso_GRAD_CAM.jpeg,Ext_Bio_0036_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Bio_0036_Verso_GUIDED_GRAD_CAM.jpeg
3,Ext_Bio_0038_Recto.jpeg,Ext_Bio_0038_Verso.jpeg,Mac,0.746596,Ext_Bio_0038_Recto_GRAD_CAM.jpeg,Ext_Bio_0038_Verso_GRAD_CAM.jpeg,Ext_Bio_0038_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Bio_0038_Verso_GUIDED_GRAD_CAM.jpeg
4,Ext_Rha_2301_Recto.jpeg,Ext_Rha_2301_Verso.jpeg,Alt,0.482000,Ext_Rha_2301_Recto_GRAD_CAM.jpeg,Ext_Rha_2301_Verso_GRAD_CAM.jpeg,Ext_Rha_2301_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Rha_2301_Verso_GUIDED_GRAD_CAM.jpeg
5,Ext_Rha_2432_Recto.jpeg,Ext_Rha_2432_Verso.jpeg,Syl,0.971778,Ext_Rha_2432_Recto_GRAD_CAM.jpeg,Ext_Rha_2432_Verso_GRAD_CAM.jpeg,Ext_Rha_2432_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Rha_2432_Verso_GUIDED_GRAD_CAM.jpeg
6,Ext_Rha_2433_Recto.jpeg,Ext_Rha_2433_Verso.jpeg,Syl,0.999177,Ext_Rha_2433_Recto_GRAD_CAM.jpeg,Ext_Rha_2433_Verso_GRAD_CAM.jpeg,Ext_Rha_2433_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Rha_2433_Verso_GUIDED_GRAD_CAM.jpeg
7,Ext_Rha_2434_Recto.jpeg,Ext_Rha_2434_Verso.jpeg,Syl,0.963252,Ext_Rha_2434_Recto_GRAD_CAM.jpeg,Ext_Rha_2434_Verso_GRAD_CAM.jpeg,Ext_Rha_2434_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Rha_2434_Verso_GUIDED_GRAD_CAM.jpeg
8,Ext_Rha_2435_Recto.jpeg,Ext_Rha_2435_Verso.jpeg,Syl,0.961535,Ext_Rha_2435_Recto_GRAD_CAM.jpeg,Ext_Rha_2435_Verso_GRAD_CAM.jpeg,Ext_Rha_2435_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Rha_2435_Verso_GUIDED_GRAD_CAM.jpeg
9,Ext_Rha_2436_Recto.jpeg,Ext_Rha_2436_Verso.jpeg,Syl,0.992694,Ext_Rha_2436_Recto_GRAD_CAM.jpeg,Ext_Rha_2436_Verso_GRAD_CAM.jpeg,Ext_Rha_2436_Recto_GUIDED_GRAD_CAM.jpeg,Ext_Rha_2436_Verso_GUIDED_GRAD_CAM.jpeg


In [30]:
df_results_ext["outputs_ext_preds"].value_counts()

Syl    283
Mac     63
Alt     29
Mil      9
Myc      6
Pse      3
Big      1
Name: outputs_ext_preds, dtype: int64

## For 